In [2]:
import pandas as pd
import numpy as np

In [3]:
# %run -i "../solve_jax.py" -m KH -L1 2 
#* you can run the python executable directly and can use the same arguments defined in the executable

In [110]:
spins = [2] * 12
groundstate = np.load("out/KH_2x2/Jx_1_Jy_1_Jz_1_h_0/groundstate.npy")
groundstate = groundstate.reshape(spins)
x = groundstate.copy()

## Check if the transition symmetry is correct

In [80]:
r1 = x.reshape(2**3, -1)
r2 = x.transpose([6,7,8,9,10,11,0,1,2,3,4,5]).reshape(2**3, -1)
partial_trace1 = np.einsum("im,jm->ij", r1, r1)
partial_trace2 = np.einsum("im,jm->ij", r2, r2)
print("difference of partial trace : ", np.linalg.norm(partial_trace1 - partial_trace2))
rho = partial_trace1
rho = (rho + rho.T)/2 #* make sure rho is Hermitian ( moreover, it should be positive semi-definite )

difference of partial trace :  1.4910943e-07


## Conduct eigen decomposition

In [91]:
E, V = np.linalg.eigh(rho)
print("sum of eigenvalues = ",E.sum()) #* should be 1
print("All eigenvalues are positive : ", (E>=0).all())
print("recreate from eigenvalues and vectors / diff : ",np.linalg.norm(V @ np.diag(E) @ V.T - rho)) #* should be zero

sum of eigenvalues =  1.0000002
All eigenvalues are positive :  True
recreate from eigenvalues and vectors / diff :  3.5450057e-08


### Check overwrap with local ground state

- #### Partical Trace for n = 6,7,8,9,10,11


In [111]:
x = groundstate.copy()
r = x.reshape(2**6, -1)
rho = np.einsum("im,jm->ij", r, r)
rho = (rho + rho.T)/2
print("check trace : ", rho.trace())

check trace :  1.0000002


#### Original Hamiltonian


H1 is local hamiltonian between triangle (0,1,2) and (3,4,5)


In [130]:
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/2.npy")
E0, V0 = np.linalg.eigh(H0)
E1, V1 = np.linalg.eigh(H1)
E2, V2 = np.linalg.eigh(H2)
for i in range(20):
    print(f"{i}th local ground state" ,V1[:,i] @ rho @ V1[:,i]) 

#### Check overwarpping with stochastic local ground state with local ground state

In [145]:
import sys
sys.path.append("/home/user/project/python/reduce_nsp")
import nsp
SH1 = -nsp.utils.stoquastic(-H1) #* make all-off-diagonal elements negative
SE1, SV1 = np.linalg.eigh(SH1)

since stoquastic local ground state can be all positive vector, and try to be the same vector up to sign of its elements, inner product of the two vectors can be good measure of optimization.

- Of course, you can compare it with their ground energy.

In [149]:
print(np.abs(SV1[:,0]) @ np.abs(V1[:,0]) )
print(abs(SE1[0]-E1[0]))

0.985526456763024
0.026504056397945175


This is really good result, which means local hamiltonian almost doesn't have negative sign. 
Next compare for Partical Diagonalized Hamiltonian

In [158]:
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3siteDiag/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3siteDiag/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3siteDiag/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/2.npy")
E0, V0 = np.linalg.eigh(H0)
E1, V1 = np.linalg.eigh(H1)
E2, V2 = np.linalg.eigh(H2)

### Optimized Hamiltonian

In [218]:
import numpy as np
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/2.npy")
u = np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/u/0.npy")

H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]

# E0, V0 = np.linalg.eigh(H0)
# E1, V1 = np.linalg.eigh(H1)
# E2, V2 = np.linalg.eigh(H2)

# SE, SV = np.linalg.eigh(SH_list[1])

for l in range(3):
    E, V = np.linalg.eigh(H_list[l])
    SE, SV = np.linalg.eigh(SH_list[l])
    print(f"local {l}th")
    print(np.abs(SV[:,0]) @ np.abs(V[:,0]), abs(SE[0]-E[0]))

local 0th
0.9999349078484887 0.00034916060574463614
local 1th
0.9999283544467124 0.0002446148999630804
local 2th
0.7072735320822696 0.0017438369396829412


Check if the Local Hamiltonian is expressed by unitary matrix

In [190]:
OH0 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
OE0, OV0 = np.linalg.eigh(OH0)
np.linalg.norm(np.kron(u,u) @ H0 @ np.kron(u,u).T - OH0)

6.213945649735919e-14

# Servey with Exact Diagonalization

In [1]:
import sys
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
import nsp
from jax_exact.core.constants import *
import numpy as np

/home/user/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Test original Hamiltonian

In [3]:

# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/2.npy")

H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]




bonds = [[0,0,2], [1,0,1], [2,0,3], [0,1,3], [1,1,0], [2,1,2], [0,2,0], [1,2,3], [2,2,1], [0,3,1], [1,3,2], [2,3,0]]
_bonds = [[], [], []]
for bond in bonds:
    _bonds[bond[0]].append(bond[1:])


#### Pure hamiltonian

In [4]:
_H = nsp.utils.sum_ham(H_list[0], _bonds[0], 4, 8)
_H += nsp.utils.sum_ham(H_list[1], _bonds[1], 4, 8)
_H += nsp.utils.sum_ham(H_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_H = jnp.array(_H)
_E, _V = jax.scipy.linalg.eigh(_H)



In [5]:
i = 0
x = _V[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,6,7,8,3,4,5,9,10,11]) # for l = 0
l = 0
r = x.reshape(2**6, -1)
rho = np.einsum("im,jm->ij", r, r)
rho = (rho + rho.T)/2
LE, LV = np.linalg.eigh(H_list[l])
print(f"l = 0 / innder product of eigenvector [{i}] : ",LV[:,i] @ rho @ LV[:,i])

x = _V[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
rho = np.einsum("im,jm->ij", r, r)
rho = (rho + rho.T)/2
LE, LV = np.linalg.eigh(H_list[l])
print(f"l = 1 / innder product of eigenvector [{i}] : ",LV[:,i] @ rho @ LV[:,i])

x = _V[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,9,10,11,3,4,5,6,7,8]) # for l = 2
l = 2
r = x.reshape(2**6, -1)
rho = np.einsum("im,jm->ij", r, r)
rho = (rho + rho.T)/2
LE, LV = np.linalg.eigh(H_list[l])
print(f"l = 2 / innder product of eigenvector [{i}] : ",LV[:,i] @ rho @ LV[:,i])

l = 0 / innder product of eigenvector [0] :  0.20229731150902824
l = 1 / innder product of eigenvector [0] :  0.20229698950424807
l = 2 / innder product of eigenvector [0] :  0.20229675550945114


The above show that global ground state is covered by local ground state to some extent.

#### Stoquastic hamiltonian

In [6]:
_SH = nsp.utils.sum_ham(SH_list[0], _bonds[0], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[1], _bonds[1], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_SH = jnp.array(_SH)
_SE, _SV = jax.scipy.linalg.eigh(_SH)

In [8]:
print("Inner product of ground state", _SV[:,0] @ jnp.abs(_V[:,0]))

Inner product of ground state -0.8145067


### Test optimized Hamiltonian

In [10]:
import sys
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
from jax_exact.core.constants import *
import numpy as np
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/2.npy")

H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]




bonds = [[0,0,2], [1,0,1], [2,0,3], [0,1,3], [1,1,0], [2,1,2], [0,2,0], [1,2,3], [2,2,1], [0,3,1], [1,3,2], [2,3,0]]
_bonds = [[], [], []]
for bond in bonds:
    _bonds[bond[0]].append(bond[1:])


#### Pure hamiltonian

In [11]:
_H = nsp.utils.sum_ham(H_list[0], _bonds[0], 4, 8)
_H += nsp.utils.sum_ham(H_list[1], _bonds[1], 4, 8)
_H += nsp.utils.sum_ham(H_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_H = jnp.array(_H)
_E, _V = jax.scipy.linalg.eigh(_H)


#### Stoquastic hamiltonian

In [12]:
_SH = nsp.utils.sum_ham(SH_list[0], _bonds[0], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[1], _bonds[1], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_SH = jnp.array(_SH)
_SE, _SV = jax.scipy.linalg.eigh(_SH)

In [13]:
print("Inner product of ground state", _SV[:,0] @ jnp.abs(_V[:,0]))

Inner product of ground state 0.4446661


Check overlapping of stoquastic local ground state with stoquastic global ground state

In [14]:
i = 0
x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,6,7,8,3,4,5,9,10,11]) # for l = 0
l = 0
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 0 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)
x = _SV[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 1 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)

x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,9,10,11,3,4,5,6,7,8]) # for l = 2
l = 2
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 2 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)


l = 0 / innder product of eigenvector [0] :  0.13976965871283176
local stoquastic energy of ground state : -1.000349160605744
--------------------
l = 1 / innder product of eigenvector [0] :  0.14017802137647267
local stoquastic energy of ground state : -1.000244614899966
--------------------
l = 2 / innder product of eigenvector [0] :  0.9984060123397892
local stoquastic energy of ground state : -1.0017438369396832
--------------------


This suggests that the wors optimized local hamiltonian amoung three behave badly, leading huge gap between global ground state energy?

### Test optimized Hamiltonian in modified scheme (take minimum of three)

In [64]:
import sys
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
from jax_exact.core.constants import *
import numpy as np
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_244/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_244/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_244/H/2.npy")

H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]




bonds = [[0,0,2], [1,0,1], [2,0,3], [0,1,3], [1,1,0], [2,1,2], [0,2,0], [1,2,3], [2,2,1], [0,3,1], [1,3,2], [2,3,0]]
_bonds = [[], [], []]
for bond in bonds:
    _bonds[bond[0]].append(bond[1:])


#### Pure hamiltonian

In [65]:
_H = nsp.utils.sum_ham(H_list[0], _bonds[0], 4, 8)
_H += nsp.utils.sum_ham(H_list[1], _bonds[1], 4, 8)
_H += nsp.utils.sum_ham(H_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_H = jnp.array(_H)
_E, _V = jax.scipy.linalg.eigh(_H)


#### Stoquastic hamiltonian

In [61]:
_SH = nsp.utils.sum_ham(SH_list[0], _bonds[0], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[1], _bonds[1], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_SH = jnp.array(_SH)
_SE, _SV = jax.scipy.linalg.eigh(_SH)

In [62]:
print("Inner product of ground state", _SV[:,0] @ jnp.abs(_V[:,0]))
_SE

Inner product of ground state -0.62793386


Array([-11.4248295, -11.421765 ,  -9.034054 , ...,   5.4970393,
         5.9149704,   5.9155645], dtype=float32)

In [63]:
for l in range(3):
    print(np.linalg.eigvalsh(SH_list[l])[0])

-1.000177700541588
-1.158498533022129
-1.0000676290110921


Check overlapping of stoquastic local ground state with stoquastic global ground state

In [55]:
i = 0
x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,6,7,8,3,4,5,9,10,11]) # for l = 0
l = 0
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 0 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)
x = _SV[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 1 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)

x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,9,10,11,3,4,5,6,7,8]) # for l = 2
l = 2
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 2 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)


l = 0 / innder product of eigenvector [0] :  0.13976965871283176
local stoquastic energy of ground state : -1.000349160605744
--------------------
l = 1 / innder product of eigenvector [0] :  0.14017802137647267
local stoquastic energy of ground state : -1.000244614899966
--------------------
l = 2 / innder product of eigenvector [0] :  0.9984060123397892
local stoquastic energy of ground state : -1.0017438369396832
--------------------


In this case, all hamiltonian optimized well. But only one of them occupy the global ground state. I expected it equally distributed among three.

### Test optimized Hamiltonian in modified scheme (take maximum of three)

In [36]:
import sys
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
from jax_exact.core.constants import *
import numpy as np
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/H/2.npy")

H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]




bonds = [[0,0,2], [1,0,1], [2,0,3], [0,1,3], [1,1,0], [2,1,2], [0,2,0], [1,2,3], [2,2,1], [0,3,1], [1,3,2], [2,3,0]]
_bonds = [[], [], []]
for bond in bonds:
    _bonds[bond[0]].append(bond[1:])


#### Pure hamiltonian

In [37]:
_H = nsp.utils.sum_ham(H_list[0], _bonds[0], 4, 8)
_H += nsp.utils.sum_ham(H_list[1], _bonds[1], 4, 8)
_H += nsp.utils.sum_ham(H_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_H = jnp.array(_H)
_E, _V = jax.scipy.linalg.eigh(_H)


#### Stoquastic hamiltonian

In [38]:
_SH = nsp.utils.sum_ham(SH_list[0], _bonds[0], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[1], _bonds[1], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_SH = jnp.array(_SH)
_SE, _SV = jax.scipy.linalg.eigh(_SH)

In [39]:
print("Inner product of ground state", _SV[:,0] @ jnp.abs(_V[:,0]))

Inner product of ground state 0.4446772


In [40]:
for l in range(3):
    print(np.linalg.eigvalsh(SH_list[l])[0])

-1.0002488650118235
-1.0004216832046366
-1.0039396389784765


Note that the result is worser than the one with optimization of average.
Check overlapping of stoquastic local ground state with stoquastic global ground state

In [41]:
i = 0
x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,6,7,8,3,4,5,9,10,11]) # for l = 0
l = 0
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 0 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)
x = _SV[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 1 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)

x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,9,10,11,3,4,5,6,7,8]) # for l = 2
l = 2
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 2 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)


l = 0 / innder product of eigenvector [0] :  0.1486917598370786
local stoquastic energy of ground state : -1.000248865011823
--------------------
l = 1 / innder product of eigenvector [0] :  0.14748093107524332
local stoquastic energy of ground state : -1.0004216832046362
--------------------
l = 2 / innder product of eigenvector [0] :  0.9984103447999974
local stoquastic energy of ground state : -1.0039396389784765
--------------------


In this case, all hamiltonian optimized well. But only one of them occupy the global ground state. I expected it equally distributed among three.

In [44]:
x = _SV[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
for i in range(10):
    print(f"l = 1 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
    print(f"local stoquastic energy of ground state : {LSE[i]}")

l = 1 / innder product of eigenvector [0] :  0.14748093107524332
local stoquastic energy of ground state : -1.0004216832046362
l = 1 / innder product of eigenvector [1] :  0.39957868359682625
local stoquastic energy of ground state : -0.9796404081959669
l = 1 / innder product of eigenvector [2] :  0.3223953965597398
local stoquastic energy of ground state : -0.9547046096658286
l = 1 / innder product of eigenvector [3] :  0.11561606151367278
local stoquastic energy of ground state : -0.922938647596009
l = 1 / innder product of eigenvector [4] :  3.311896112410876e-05
local stoquastic energy of ground state : -0.8478330584558101
l = 1 / innder product of eigenvector [5] :  8.254510672067024e-05
local stoquastic energy of ground state : -0.8276318741800783
l = 1 / innder product of eigenvector [6] :  7.667640706862021e-05
local stoquastic energy of ground state : -0.8038576974393363
l = 1 / innder product of eigenvector [7] :  2.490896607239086e-05
local stoquastic energy of ground state 

This suggest global ground state is mostly occupied by excited state rather than ground state.

v